In [96]:
import csv
import random
import argparse
import math
import pickle
import os
import torch
import numpy as np
import timeit
import torch
import time
import pickle
from helper.dataloader import Dataset_with_mine_EC, Dataset_lookup
from model import Net
from helper.utils import get_ec_id_dict, mine_hard_negative, parse, seed_everything
import torch.nn as nn
from helper.distance_map import get_dist_map

dtype = torch.float32; device = torch.device("cuda")

In [36]:
args_model_name = "uniref10_train_split_0"
args_hidden_dim = 512
args_out_dim  = 128
id_ec, ec_id_dict = get_ec_id_dict('./data/' + args_model_name + '.csv')
ec_id = {key: list(ec_id_dict[key]) for key in ec_id_dict.keys()}
dist_map = pickle.load( open('./data/distance_map/' + args_model_name + '.pkl', 'rb'))
model = Net(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_train_split_0_50.pth')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [82]:
def get_dist_map(out_dim, file_name, model=None):
    
    params = {'batch_size': 30000,
              'shuffle': False,
              'num_workers': 8}

    counter = 0
    cluster_center_esm = {}
    with torch.no_grad():
        _, ec_id_dict = get_ec_id_dict('./data/' + file_name + '.csv')
        time_start = time.time()
        total_ec_n = len(ec_id_dict.keys())

        for ec in list(ec_id_dict.keys()):
            # TODO: this if statement might be legacy code?
            if ec.count('-') == 0:
                counter += 1
                if counter > 10 : break
                valid_data = Dataset_lookup(list(ec_id_dict[ec]))
                valid_loader = torch.utils.data.DataLoader(
                    valid_data, **params)
                for i, data in enumerate(valid_loader):
                    if model != None:
                        data = data.to(device=model.device, dtype=model.dtype)
                        print(data.size())
                        lookup = model(data)
                    esm = torch.mean(lookup, dim=0)
                    cluster_center_esm[ec] = esm.detach().cpu()
                if counter % 2 == 0:
                    elapsed = time.time() - time_start
                    time_start = time.time()
                    print(
                        f'|Processing {counter:5d} out of {total_ec_n} |Time elapsed: {elapsed:5.2f} |')

        esm_lookup = torch.randn(total_ec_n, out_dim)
        ecs = list(cluster_center_esm.keys())
        for i, ec in enumerate(ecs):
            esm_lookup[i] = cluster_center_esm[ec]
        esm_lookup = esm_lookup.to(device)
        esm_dist = {}
        time_start = time.time()
        for i, ec in enumerate(ecs):
            current = esm_lookup[i]
            current = current.repeat(total_ec_n, 1)
            current = current.to(device)
            norm_esm = (current - esm_lookup).norm(dim=1, p=2)
            norm_esm = norm_esm.detach().cpu().numpy()
            esm_dist[ec] = {}
            for j, k in enumerate(ecs):
                esm_dist[ec][k] = norm_esm[j]
            if i % 2 == 0:
                elapsed = time.time() - time_start
                time_start = time.time()
                print(
                    f'|Processing {i:5d} out of {total_ec_n} |Time elapsed: {elapsed:5.5f} |')
        return esm_dist

In [69]:
def format(a):
    if type(a) == dict:
        a = a['mean_representations'][33]
    return a

In [81]:
N_query = 0
for ec in list(ec_id_dict.keys()):
    N_query += len(ec_id_dict[ec])
N_query   

25608

In [71]:
for ec in list(ec_id_dict.keys()):
    print(ec_id_dict[ec])
    break
lookup = list(ec_id_dict[ec])[0]
esm_lookup = format(torch.load('./data/esm_data/'+ lookup + '.pt'))
print("the first id in first ec is:", lookup)
print("its esm is:", esm_lookup)

{'Q48743', 'Q8BL86', 'D4AXL1', 'Q59514', 'P52700', 'P30898', 'Q9ZMJ9', 'P0A3M4', 'P96465', 'P67920', 'P0A1V8', 'P52664', 'P14559', 'Q00982', 'P52682', 'P30899', 'O07293', 'B3U538', 'P04190', 'Q9K303', 'P45460', 'Q44056', 'O24968', 'B0V2S2', 'O25103', 'P26918', 'P14171', 'P13661', 'P37321', 'Q00983', 'P52699', 'P71778', 'C7C422', 'A5U493', 'Q59517', 'P81173', 'P16897', 'P24735', 'Q5YXD6'}
the first id in first ec is: Q48743
its esm is: tensor([ 0.1186,  0.2027,  0.0066,  ..., -0.0034, -0.0675,  0.1768])


In [98]:
# model(esm_lookup.to(device))
esm_dummy = esm_lookup.unsqueeze(0) 
esm_dummy = esm_dummy.repeat(25608, 1) 
%timeit model(esm_dummy.to(device)).size()

15.2 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
dist_map = get_dist_map(args_out_dim, args_model_name, model=model)

torch.Size([39, 1280])
torch.Size([1, 1280])
|Processing     2 out of 4209 |Time elapsed:  2.02 |
torch.Size([312, 1280])
torch.Size([15, 1280])
|Processing     4 out of 4209 |Time elapsed:  2.21 |
torch.Size([11, 1280])
torch.Size([13, 1280])
|Processing     6 out of 4209 |Time elapsed:  2.04 |
torch.Size([7, 1280])
torch.Size([23, 1280])
|Processing     8 out of 4209 |Time elapsed:  2.02 |
torch.Size([1, 1280])
torch.Size([15, 1280])
|Processing    10 out of 4209 |Time elapsed:  1.99 |
|Processing     0 out of 4209 |Time elapsed: 0.00050 |
|Processing     2 out of 4209 |Time elapsed: 0.00050 |
|Processing     4 out of 4209 |Time elapsed: 0.00050 |
|Processing     6 out of 4209 |Time elapsed: 0.00050 |
|Processing     8 out of 4209 |Time elapsed: 0.00000 |
